##### Charger les poids du modèle
##### model.load_state_dict(torch.load('/kaggle/working/model_weights_Resnet50.pth'))

In [ ]:
custum_test_loader = DataLoader(custum_test_dataset, batch_size=32, shuffle=False, num_workers=4)
# Phase de test
model.eval()
y_pred=[]
y_true=[]
val_loss_items = []
correct_val = 0
total_val = 0
val_tqdm = tqdm(custum_test_loader, desc='test', leave=False)
with torch.no_grad():
    for images, labels,name in val_tqdm:
#         images = images.to(device)  # Envoyer les images au GPU
#         labels = labels.to(device)  # Envoyer les étiquettes au GPU
        images, labels = images.cuda(), labels.cuda()

        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss_items.append(loss.item())
        _, predicted = torch.max(outputs.data, 1)
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()
        current_val_accuracy = 100 * correct_val / total_val
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

        val_tqdm.set_postfix(loss=loss.item(), accuracy=f'{current_val_accuracy:.2f}%')

avg_val_loss = sum(val_loss_items) / len(val_loss_items)
print(f'Test Loss: {avg_val_loss}, Test Accuracy: {current_val_accuracy:.2f}%')

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print(conf_matrix)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.title('Matrice de Confusion')
plt.show()

In [ ]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.files = []
        self.names = []
        self.failed_count=0
        study_tqdm = tqdm(os.listdir(test_path), desc=f'Creation du dataset...', leave=True)
        for study_id in study_tqdm:
            study_folder_path = os.path.join(test_path, study_id)
            
            if os.path.isdir(study_folder_path):
                # Parcourir chaque dossier de série dans le dossier study_id
                for series_id in os.listdir(study_folder_path):
                    series_folder_path = os.path.join(study_folder_path, series_id)
                    
                    if os.path.isdir(series_folder_path):
                        # Compter le nombre de fichiers .dcm dans chaque dossier de série
                        for f in os.listdir(series_folder_path):
                            if f.endswith('.dcm'):
                                filter_df_label=df_label[(df_label["study_id"]==int(study_id)) & (df_label["series_id"]==int(series_id)) & (df_label["instance_number"]==int(f.replace(".dcm","")))]
                                for index, row in filter_df_label.iterrows():
                                    file_name=row['condition'].replace(" ","_")+"_"+row['level'].replace("/","_")
                                    file_name=file_name.lower()
                                    self.names.append(study_id+"_"+file_name)
                                    self.files.append(os.path.join(series_folder_path, f)) 

        self.transform = transform

        if len(self.files) == 0:
            raise ValueError("No image (dcm) files found in the specified directory.")
#
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        dicom_data = pydicom.dcmread(dcm_file_path)

        image = dicom_data.pixel_array
        image = image.astype(np.float32)
        
        if self.transform:
            image = self.transform(image)
        
        name=self.names[idx]
        return image,name


In [ ]:
# Nous allons tester la classe TestDataset
test_dataset = TestDataset(test_path, transform=test_transform) # ici je transforme les images en Tensor pour une utilisation rapide de PyTorch Dataloader

# on aura besoin d'un dataloader qui enveloppe notre objet test_dataset
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4) # souvent le shuffle est mis a False, mais avec notre implementation, ca marche dans tous les cas

In [ ]:
# generons une prediction aleatoire pour notre test
model.to(device)
model.eval()
normal = []
modere = []
severe = []
image_names = []

for batch,im_names in test_loader:
    images=batch.to(device)
    with torch.no_grad():
        outputs = model(images)
        outputs = torch.softmax(outputs,1)
        print(outputs)
        #_, predicted = torch.max(outputs, 1)
        normal.extend(outputs[0])
        modere.extend(outputs[1])
        severe.extend(outputs[2])

    # retenir les noms des images
    image_names.extend(im_names)

assert len(modere) == len(image_names)
assert len(modere) == len(test_dataset) # est-ce qu'on a prédits tous les exemples de tests ?
print(f'Il y a {len(label_predictions)} exemples de test')

In [ ]:
# Utilisons Pandas afin de generer un DataFrame

predictions_df = pd.DataFrame(data=zip(image_names, label_predictions), columns=['row_id', 'class_label'])
predictions_df